In [6]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import os.path
import json
import pickle
import h5py
import sys
import F1F0

from var_utils import *

save_result_dir = r"./"
dir_syn = r"./biophysical"
# dir_stp = r"//allen/programs/braintv/workgroups/v1column/binghuangc/network_effect/Column_Simulation/STP_update_syn_tau/run_spike_trains/results/"


# Load the nodes
nodes_DF = pd.read_csv('../Biophysical_network/net/v1_nodes.csv', sep = ' ')
print "Neuron dataframe:"
print nodes_DF.head()

orientations = np.arange(0,360,45)
trials = np.arange(10)
stimulusTypeName = "gratings"
total_n_neurons = 230924
np.random.seed(100)

pop_names_bph = np.array(['e23Cux2', 'e4Rorb', 'e4Nr5a1' , 'e4Scnn1a', 'e4other','e5Rbp4','e5noRbp4', 'e6Ntsr1'
                , 'i23Pvalb', 'i1Htr3a','i23Htr3a','i23Sst', 'i4Pvalb', 'i4Htr3a', 'i4Sst', 'i5Pvalb'
                 , 'i5Sst', 'i5Htr3a', 'i6Pvalb', 'i6Sst', 'i6Htr3a'])

Neuron dataframe:
   id   model_id      x_soma      y_soma      z_soma  rotation_angle_yaxis  \
0   0  100000101  110.906706 -431.668136  623.732700              1.247325   
1   1  100000101  526.329919 -564.693981 -555.341582              3.418240   
2   2  100000101  197.897567 -505.737886  709.692990              0.463397   
3   3  100000101  400.302561 -465.458859 -224.639993              4.102361   
4   4  100000101 -766.751467 -556.178476   55.929619              2.800441   

    pop_name ei location  tuning_angle  
0  LIFe5Rbp4  e    VisL5      0.000000  
1  LIFe5Rbp4  e    VisL5      0.017278  
2  LIFe5Rbp4  e    VisL5      0.034556  
3  LIFe5Rbp4  e    VisL5      0.051833  
4  LIFe5Rbp4  e    VisL5      0.069111  


In [7]:
def bin_resp_spk(spike_data, n_neurons, start_time, end_time, win_size, bin_stride):
    # assumes spike_data DataFrame
    spike_rate = np.zeros((n_neurons,len(range(start_time, end_time, bin_stride))))           
    for l_ind, t_ind in enumerate(range(start_time,end_time, bin_stride)):
        count_loop = spike_data[spike_data["time"]>t_ind][spike_data["time"]<t_ind+win_size].groupby("cell")["time"].count()
        spike_rate[count_loop.index,l_ind] = count_loop
    #print "Done!"
    return spike_rate

def sig_cor_across_pop(total_spike_count1, total_spike_count2, remove_nans):
    # assumes total_spike_count has size m*n*r where m is number of neurons, n is number of orientations
    # and r is number of trials
    # assumes remove_nans is a binary flag
    
    spike_corrs_exc = []
    
    trial_ave_spike_exc1 = np.nanmean(total_spike_count1, axis = 2)
    trial_ave_spike_exc2 = np.nanmean(total_spike_count2, axis = 2)
    
    for i in range(trial_ave_spike_exc1.shape[0]):
        for j in range(trial_ave_spike_exc2.shape[0]):
            spike_corrs_exc.append(np.corrcoef((trial_ave_spike_exc1[i,:],trial_ave_spike_exc2[j,:]))[0,1])
        
    tempR_exc = np.array(spike_corrs_exc)
    print tempR_exc.shape
#     tempR_exc = np.tril(spike_corrs_exc)
#     tempR_exc = tempR_exc[np.tril_indices(int(np.sqrt(tempR_exc.size)), -1)]
    if remove_nans:
        tempR_exc = tempR_exc[~np.isnan(tempR_exc)]        
    return tempR_exc

def change_bin_win(spike_rate_ephys_1ms, win_size):
    spike_rate_ephys = np.zeros((spike_rate_ephys_1ms.shape[0], np.int(spike_rate_ephys_1ms.shape[1]/win_size)+1, spike_rate_ephys_1ms.shape[2]))
    for bin_ind, bin_edge in enumerate(range(0,spike_rate_ephys_1ms.shape[1],win_size)):
        spike_rate_ephys[:,bin_ind,:] = np.sum(spike_rate_ephys_1ms[:,bin_edge:bin_edge+win_size,:], axis=1)
    return spike_rate_ephys

In [8]:
# parameters for calculation of firing rate over step

flash_types = ['on', 'off']

win_size = 10
bin_stride = 10

for flash_type in flash_types:
    
    if flash_type == 'on':
        end_time = 750
        start_time = 500
        
    elif flash_type == 'off':
        end_time = 2000
        start_time = 1750

    spike_rates = np.zeros((total_n_neurons, (end_time - start_time)/bin_stride, trials.shape[0]))

    for t in trials:
        if stimulusTypeName == "gratings":
            run_name = 'output_trial'+str(t)+'/spikes.txt'
    #                 spikes = np.loadtxt(dir_syn+spikes_file_name, unpack=True)
    #                 run_name =  'full3_GScorrected_PScorrected_3.0sec_SF0.04_TF2.0_ori'+str(o)+'.0_c80.0_gs0.5_spikes/trial_'+str(t)+'/output/spikes.txt'
            spike_file = os.path.join(dir_syn, run_name)
            spike_data = pd.read_table(spike_file, sep=" ", names=["time", "cell"])
    #                 spike_data = spike_data.ix[spike_data["cell"] < total_n_neurons, :] # don't care about LIFs
            spike_data = spike_data.ix[spike_data["time"] > 0, :] # remove the spont part
            spike_rates[:,:,t] = spike_rates[:,:,t] + bin_resp_spk(spike_data, total_n_neurons, start_time, end_time, win_size, bin_stride) 
        print "Data loaded for trial: ", t

    np.save(save_result_dir+'spike_rates_250msFlashes_'+flash_type+'.npy', spike_rates)
    

Data loaded for trial:  0
Data loaded for trial:  1
Data loaded for trial:  2
Data loaded for trial:  3
Data loaded for trial:  4
Data loaded for trial:  5
Data loaded for trial:  6
Data loaded for trial:  7
Data loaded for trial:  8
Data loaded for trial:  9
Data loaded for trial:  0
Data loaded for trial:  1
Data loaded for trial:  2
Data loaded for trial:  3
Data loaded for trial:  4
Data loaded for trial:  5
Data loaded for trial:  6
Data loaded for trial:  7
Data loaded for trial:  8
Data loaded for trial:  9


#### Need to make directories for saving

In [9]:
flash_types = ['on', 'off']
connection_types = ['ee', 'ii', 'all']

np.random.seed(0)
tot_rand = 1000

inds_e_all = np.where(nodes_DF.pop_name.str.startswith('e'))[0]
print inds_e_all.shape
inds_e = np.random.choice(inds_e_all, tot_rand, replace=False)

inds_i_all = np.where(nodes_DF.pop_name.str.startswith('i'))[0]
print inds_i_all.shape
inds_i = np.random.choice(inds_i_all, tot_rand, replace=False)

for flash_type in flash_types:
    
    total_spike_count = np.load(save_result_dir+'spike_rates_250msFlashes_'+flash_type+'.npy')

    for connection_type in connection_types:
        
        if connection_type == 'ee':
            total_spike_count_pop = total_spike_count[inds_e,:,:]
            print 'total number of neurons: ', total_spike_count_pop.shape[0]
            
        elif connection_type == 'ii':
            total_spike_count_pop = total_spike_count[inds_i,:,:]
            print 'total number of neurons: ', total_spike_count_pop.shape[0]
            
        elif connection_type == 'all':
            total_spike_count_pop = total_spike_count[np.union1d(inds_e,inds_i),:,:]
            print 'total number of neurons: ', total_spike_count_pop.shape[0]

        sig_cors = sig_cor(total_spike_count_pop, 0)
        print 'Signal correlation Done!'
    #     noise_cors = noise_cor(total_spike_count_pop, 0)
    #     print 'Noise correlation Done!'

    #     sig_noise_cors = sig_noise_cor(sig_cors,noise_cors)
    # #         plot_sig_noise_cor(tempR_sig,tempR_noise)

        np.savez(save_result_dir+'corrs/250msflashes/biophysical/corrs_250msFlashes_'+flash_type+'_'+connection_type+'.npz', sig_cors = sig_cors)
        
#ei
for flash_type in flash_types:
    
    total_spike_count = np.load(save_result_dir+'spike_rates_250msFlashes_'+flash_type+'.npy')
    sig_cors = sig_cor_across_pop(total_spike_count[inds_e,:,:], total_spike_count[inds_i,:,:], 0)
    print 'Signal correlation Done!'
    
    np.savez(save_result_dir+'corrs/250msflashes/biophysical/corrs_250msFlashes_'+flash_type+'_ei.npz', sig_cors = sig_cors)


(43394,)
(8584,)
total number of neurons:  1000
(1000, 1000)
Signal correlation Done!
total number of neurons:  1000
(1000, 1000)
Signal correlation Done!
total number of neurons:  2000
(2000, 2000)
Signal correlation Done!
total number of neurons:  1000
(1000, 1000)
Signal correlation Done!
total number of neurons:  1000
(1000, 1000)
Signal correlation Done!
total number of neurons:  2000
(2000, 2000)
Signal correlation Done!
(1000000,)
Signal correlation Done!
(1000000,)
Signal correlation Done!
